<h1 style='font-size:20px;text-align: center'>How to store MODIS, Caliop and CloudSat co-located indexes in a HDF5 file ?</h1>

<p style='text-align: center'>
Author: Benjamin Marchant 
<br>(marchant.benjamin01@gmail.com)
<br>(benjamin.marchant@nasa.gov)
<br>
<br> Last update: 2021-01-19
</p>

Table of Content:

* [Download Caliop 1km Cloud Layer from Icare server](#download_caliop_1km_cloud_layer)
* [Download Caliop 5km Cloud Layer from Icare server](#download_caliop_5km_cloud_layer)
* [Download CloudSat 2B-GEOPROF from Icare server](#download_cloudsat_geoprof)
* [Download MODIS MYD03 from NASA LADSWEB](#download_modis_myd03)
* [Create a function to store colocated indexes in a HDF5 file](#create_hdf5_file)
* [Read data from the HDF5 file](#read_hdf5)

In [5]:
from ftplib import FTP
from matplotlib.pyplot import figure
from pyhdf.SD import SD, SDC 
from scipy.spatial import distance_matrix
from scipy.spatial.distance import pdist, cdist
from pyhdf.HDF import *
from pyhdf.VS import *
from os import path
from datetime import date
from calendar import monthrange

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import seaborn as sns; sns.set()
import datetime 
import urllib.request
import urllib.request, json 
import pandas as pd
import os
import calendar
import numpy as np
import glob
import re
import pprint
import warnings

warnings.filterwarnings('ignore')

In [6]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

In [13]:
# enter the path where the data will be stored

path_to_media = '/Volumes/HD1/media/'

In [17]:
# enter icare username and password

icare_username = '**********'
icare_password = '**********'

In [22]:
# enter token to access NASA LADS

opener = urllib.request.build_opener()
opener.addheaders = [('Authorization', 'Bearer ***********')]
urllib.request.install_opener(opener)

### Download MODIS MYD03 from NASA LADSWEB <a class="anchor" id="download_modis_myd03"></a>

In [23]:
# test 1 day

year = 2014
month = 2
day = 1

os.system( 'mkdir {}MODIS'.format(path_to_media) )
os.system( 'mkdir {}MODIS/MYD03'.format(path_to_media) )
os.system( 'mkdir {}MODIS/MYD03/{:04d}'.format(path_to_media,year) )
os.system( 'mkdir {}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}'.format(path_to_media,year,year,month,day) )

0

In [24]:
d0 = date(year, 1, 1)
d1 = date(year, month, day)
delta = d1 - d0

count_of_day = delta.days + 1

print(count_of_day)

32


In [27]:
%%time

ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/{:04d}/{:03d}.json'.format(year,count_of_day)

with urllib.request.urlopen(ladsweb_url) as url:
    data = json.loads(url.read().decode())

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(data)

[   {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0000.061.2018051152159.hdf',
        'size': 32955175},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0005.061.2018051151531.hdf',
        'size': 29229726},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0010.061.2018051151021.hdf',
        'size': 27494501},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0015.061.2018051151148.hdf',
        'size': 27329534},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0020.061.2018051151248.hdf',
        'size': 25490213},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0025.061.2018051151033.hdf',
        'size': 25843697},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A2014032.0030.061.2018051151212.hdf',
        'size': 26223871},
    {   'last-modified': '2018-06-07 23:19',
        'name': 'MYD03.A

In [28]:
%%time

for file in data:
    file_name = file['name']
    print(file_name) 
    if not path.exists('{}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)):
        try:
            ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/{:04d}/{:03d}/{}'.format(year,count_of_day,file_name)
            target_dir = '{}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)
            urllib.request.urlretrieve(ladsweb_url,target_dir) 
        except:
            pass

MYD03.A2014032.0000.061.2018051152159.hdf
MYD03.A2014032.0005.061.2018051151531.hdf
MYD03.A2014032.0010.061.2018051151021.hdf
MYD03.A2014032.0015.061.2018051151148.hdf
MYD03.A2014032.0020.061.2018051151248.hdf
MYD03.A2014032.0025.061.2018051151033.hdf
MYD03.A2014032.0030.061.2018051151212.hdf
MYD03.A2014032.0035.061.2018051151009.hdf
MYD03.A2014032.0040.061.2018051151143.hdf
MYD03.A2014032.0045.061.2018051151051.hdf
MYD03.A2014032.0050.061.2018051152019.hdf
MYD03.A2014032.0055.061.2018051151157.hdf
MYD03.A2014032.0100.061.2018051150914.hdf
MYD03.A2014032.0105.061.2018051151112.hdf
MYD03.A2014032.0110.061.2018051150802.hdf
MYD03.A2014032.0115.061.2018051150841.hdf
MYD03.A2014032.0120.061.2018051150857.hdf
MYD03.A2014032.0125.061.2018051150749.hdf
MYD03.A2014032.0130.061.2018051151245.hdf
MYD03.A2014032.0135.061.2018051151824.hdf
MYD03.A2014032.0140.061.2018051152726.hdf
MYD03.A2014032.0145.061.2018051151503.hdf
MYD03.A2014032.0150.061.2018051151229.hdf
MYD03.A2014032.0155.061.2018051151

MYD03.A2014032.1620.061.2018051151516.hdf
MYD03.A2014032.1625.061.2018051152254.hdf
MYD03.A2014032.1630.061.2018051152507.hdf
MYD03.A2014032.1635.061.2018051151816.hdf
MYD03.A2014032.1640.061.2018051151248.hdf
MYD03.A2014032.1645.061.2018051151257.hdf
MYD03.A2014032.1650.061.2018051151241.hdf
MYD03.A2014032.1655.061.2018051151539.hdf
MYD03.A2014032.1700.061.2018051151630.hdf
MYD03.A2014032.1705.061.2018051151331.hdf
MYD03.A2014032.1710.061.2018051151306.hdf
MYD03.A2014032.1715.061.2018051151822.hdf
MYD03.A2014032.1720.061.2018051152011.hdf
MYD03.A2014032.1725.061.2018051151257.hdf
MYD03.A2014032.1730.061.2018051151029.hdf
MYD03.A2014032.1735.061.2018051151032.hdf
MYD03.A2014032.1740.061.2018051151142.hdf
MYD03.A2014032.1745.061.2018051151010.hdf
MYD03.A2014032.1750.061.2018051151548.hdf
MYD03.A2014032.1755.061.2018051151026.hdf
MYD03.A2014032.1800.061.2018051151700.hdf
MYD03.A2014032.1805.061.2018051152450.hdf
MYD03.A2014032.1810.061.2018051152300.hdf
MYD03.A2014032.1815.061.2018051151

In [ ]:
%%time

year = 2014
month = 2

for day in range(monthrange(year, month)[1]):
    
    day = day + 1
    
    d0 = date(year, 1, 1)
    d1 = date(year, month, day)
    delta = d1 - d0

    count_of_day = delta.days + 1
    
    os.system( 'mkdir {}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}'.format(path_to_media,year,year,month,day) )

    ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/{:04d}/{:03d}.json'.format(year,count_of_day)

    with urllib.request.urlopen(ladsweb_url) as url:
        data = json.loads(url.read().decode())    
    
    for file in data:
        file_name = file['name']
        print(file_name) 
        if not path.exists('{}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)):
            try:
                ladsweb_url = 'https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MYD03/{:04d}/{:03d}/{}'.format(year,count_of_day,file_name)
                target_dir = '{}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)
                urllib.request.urlretrieve(ladsweb_url,target_dir) 
            except:
                pass
            
    print('All file from day {} have been downloaded'.format(day))

MYD03.A2014032.0000.061.2018051152159.hdf
MYD03.A2014032.0005.061.2018051151531.hdf
MYD03.A2014032.0010.061.2018051151021.hdf
MYD03.A2014032.0015.061.2018051151148.hdf
MYD03.A2014032.0020.061.2018051151248.hdf
MYD03.A2014032.0025.061.2018051151033.hdf
MYD03.A2014032.0030.061.2018051151212.hdf
MYD03.A2014032.0035.061.2018051151009.hdf
MYD03.A2014032.0040.061.2018051151143.hdf
MYD03.A2014032.0045.061.2018051151051.hdf
MYD03.A2014032.0050.061.2018051152019.hdf
MYD03.A2014032.0055.061.2018051151157.hdf
MYD03.A2014032.0100.061.2018051150914.hdf
MYD03.A2014032.0105.061.2018051151112.hdf
MYD03.A2014032.0110.061.2018051150802.hdf
MYD03.A2014032.0115.061.2018051150841.hdf
MYD03.A2014032.0120.061.2018051150857.hdf
MYD03.A2014032.0125.061.2018051150749.hdf
MYD03.A2014032.0130.061.2018051151245.hdf
MYD03.A2014032.0135.061.2018051151824.hdf
MYD03.A2014032.0140.061.2018051152726.hdf
MYD03.A2014032.0145.061.2018051151503.hdf
MYD03.A2014032.0150.061.2018051151229.hdf
MYD03.A2014032.0155.061.2018051151

MYD03.A2014033.0000.061.2018051151228.hdf
MYD03.A2014033.0005.061.2018051151356.hdf
MYD03.A2014033.0010.061.2018051151158.hdf
MYD03.A2014033.0015.061.2018051151403.hdf
MYD03.A2014033.0020.061.2018051151142.hdf
MYD03.A2014033.0025.061.2018051151145.hdf
MYD03.A2014033.0030.061.2018051151439.hdf
MYD03.A2014033.0035.061.2018051151643.hdf
MYD03.A2014033.0040.061.2018051152637.hdf
MYD03.A2014033.0045.061.2018051152419.hdf
MYD03.A2014033.0050.061.2018051152211.hdf
MYD03.A2014033.0055.061.2018051151824.hdf
MYD03.A2014033.0100.061.2018051151442.hdf
MYD03.A2014033.0105.061.2018051151656.hdf
MYD03.A2014033.0110.061.2018051151426.hdf
MYD03.A2014033.0115.061.2018051151501.hdf
MYD03.A2014033.0120.061.2018051151351.hdf
MYD03.A2014033.0125.061.2018051151413.hdf
MYD03.A2014033.0130.061.2018051151947.hdf
MYD03.A2014033.0135.061.2018051151948.hdf
MYD03.A2014033.0140.061.2018051151601.hdf
MYD03.A2014033.0145.061.2018051151337.hdf
MYD03.A2014033.0150.061.2018051151304.hdf
MYD03.A2014033.0155.061.2018051151

MYD03.A2014033.1620.061.2018051152724.hdf
MYD03.A2014033.1625.061.2018051152014.hdf
MYD03.A2014033.1630.061.2018051151454.hdf
MYD03.A2014033.1635.061.2018051151449.hdf
MYD03.A2014033.1640.061.2018051151432.hdf
MYD03.A2014033.1645.061.2018051151626.hdf
MYD03.A2014033.1650.061.2018051151740.hdf
MYD03.A2014033.1655.061.2018051151641.hdf
MYD03.A2014033.1700.061.2018051151919.hdf
MYD03.A2014033.1705.061.2018051151906.hdf
MYD03.A2014033.1710.061.2018051152940.hdf
MYD03.A2014033.1715.061.2018051152603.hdf
MYD03.A2014033.1720.061.2018051152003.hdf
MYD03.A2014033.1725.061.2018051151851.hdf
MYD03.A2014033.1730.061.2018051151810.hdf
MYD03.A2014033.1735.061.2018051151625.hdf
MYD03.A2014033.1740.061.2018051151854.hdf
MYD03.A2014033.1745.061.2018051151831.hdf
MYD03.A2014033.1750.061.2018051151737.hdf
MYD03.A2014033.1755.061.2018051151637.hdf
MYD03.A2014033.1800.061.2018051152427.hdf
MYD03.A2014033.1805.061.2018051151518.hdf
MYD03.A2014033.1810.061.2018051151801.hdf
MYD03.A2014033.1815.061.2018051151

MYD03.A2014034.0835.061.2018051151848.hdf
MYD03.A2014034.0840.061.2018051152111.hdf
MYD03.A2014034.0845.061.2018051152015.hdf
MYD03.A2014034.0850.061.2018051152415.hdf
MYD03.A2014034.0855.061.2018051152004.hdf
MYD03.A2014034.0900.061.2018051152040.hdf
MYD03.A2014034.0905.061.2018051151734.hdf
MYD03.A2014034.0910.061.2018051151838.hdf
MYD03.A2014034.0915.061.2018051151821.hdf
MYD03.A2014034.0920.061.2018051151808.hdf
MYD03.A2014034.0925.061.2018051151709.hdf
MYD03.A2014034.0930.061.2018051151938.hdf
MYD03.A2014034.0935.061.2018051152353.hdf
MYD03.A2014034.0940.061.2018051153849.hdf
MYD03.A2014034.0945.061.2018051152511.hdf
MYD03.A2014034.0950.061.2018051152017.hdf
MYD03.A2014034.0955.061.2018051152025.hdf
MYD03.A2014034.1000.061.2018051151848.hdf
MYD03.A2014034.1005.061.2018051151958.hdf
MYD03.A2014034.1010.061.2018051151902.hdf
MYD03.A2014034.1015.061.2018051151852.hdf
MYD03.A2014034.1020.061.2018051151842.hdf
MYD03.A2014034.1025.061.2018051151857.hdf
MYD03.A2014034.1030.061.2018051153

MYD03.A2014035.0050.061.2018051152404.hdf
MYD03.A2014035.0055.061.2018051152433.hdf
MYD03.A2014035.0100.061.2018051152141.hdf
MYD03.A2014035.0105.061.2018051152104.hdf
MYD03.A2014035.0110.061.2018051152058.hdf
MYD03.A2014035.0115.061.2018051152109.hdf
MYD03.A2014035.0120.061.2018051153009.hdf
MYD03.A2014035.0125.061.2018051151945.hdf
MYD03.A2014035.0130.061.2018051151921.hdf
MYD03.A2014035.0135.061.2018051152107.hdf
MYD03.A2014035.0140.061.2018051152412.hdf
MYD03.A2014035.0145.061.2018051152119.hdf
MYD03.A2014035.0150.061.2018051151938.hdf
MYD03.A2014035.0155.061.2018051152202.hdf
MYD03.A2014035.0200.061.2018051152337.hdf
MYD03.A2014035.0205.061.2018051152447.hdf
MYD03.A2014035.0210.061.2018051153609.hdf
MYD03.A2014035.0215.061.2018051152634.hdf
MYD03.A2014035.0220.061.2018051152242.hdf
MYD03.A2014035.0225.061.2018051152207.hdf
MYD03.A2014035.0230.061.2018051152125.hdf
MYD03.A2014035.0235.061.2018051152728.hdf
MYD03.A2014035.0240.061.2018051152253.hdf
MYD03.A2014035.0245.061.2018051152

...

MYD03.A2014040.1105.061.2018051173733.hdf
MYD03.A2014040.1110.061.2018051173441.hdf
MYD03.A2014040.1115.061.2018051173140.hdf
MYD03.A2014040.1120.061.2018051173141.hdf
MYD03.A2014040.1125.061.2018051173636.hdf
MYD03.A2014040.1130.061.2018051174010.hdf
MYD03.A2014040.1135.061.2018051173147.hdf
MYD03.A2014040.1140.061.2018051172905.hdf
MYD03.A2014040.1145.061.2018051173403.hdf
MYD03.A2014040.1150.061.2018051173142.hdf
MYD03.A2014040.1155.061.2018051173218.hdf
MYD03.A2014040.1200.061.2018051173455.hdf
MYD03.A2014040.1205.061.2018051173125.hdf
MYD03.A2014040.1210.061.2018051172905.hdf
MYD03.A2014040.1215.061.2018051173503.hdf
MYD03.A2014040.1220.061.2018051173917.hdf
MYD03.A2014040.1225.061.2018051173722.hdf
MYD03.A2014040.1230.061.2018051173601.hdf
MYD03.A2014040.1235.061.2018051173554.hdf
MYD03.A2014040.1240.061.2018051173600.hdf
MYD03.A2014040.1245.061.2018051173202.hdf
MYD03.A2014040.1250.061.2018051173144.hdf
MYD03.A2014040.1255.061.2018051173631.hdf
MYD03.A2014040.1300.061.2018051173

MYD03.A2014041.0320.061.2018051153103.hdf
MYD03.A2014041.0325.061.2018051153027.hdf
MYD03.A2014041.0330.061.2018051152754.hdf
MYD03.A2014041.0335.061.2018051152953.hdf
MYD03.A2014041.0340.061.2018051152854.hdf
MYD03.A2014041.0345.061.2018051152829.hdf
MYD03.A2014041.0350.061.2018051152856.hdf
MYD03.A2014041.0355.061.2018051152858.hdf
MYD03.A2014041.0400.061.2018051153707.hdf
MYD03.A2014041.0405.061.2018051153019.hdf
MYD03.A2014041.0410.061.2018051152847.hdf
MYD03.A2014041.0415.061.2018051152803.hdf
MYD03.A2014041.0420.061.2018051152846.hdf
MYD03.A2014041.0425.061.2018051152957.hdf
MYD03.A2014041.0430.061.2018051153015.hdf
MYD03.A2014041.0435.061.2018051152840.hdf
MYD03.A2014041.0440.061.2018051152739.hdf
MYD03.A2014041.0445.061.2018051153238.hdf
MYD03.A2014041.0450.061.2018051153805.hdf
MYD03.A2014041.0455.061.2018051153525.hdf
MYD03.A2014041.0500.061.2018051152935.hdf
MYD03.A2014041.0505.061.2018051152859.hdf
MYD03.A2014041.0510.061.2018051152932.hdf
MYD03.A2014041.0515.061.2018051152

MYD03.A2014041.1940.061.2018051154519.hdf
MYD03.A2014041.1945.061.2018051153512.hdf
MYD03.A2014041.1950.061.2018051153517.hdf
MYD03.A2014041.1955.061.2018051153157.hdf
MYD03.A2014041.2000.061.2018051153146.hdf
MYD03.A2014041.2005.061.2018051153049.hdf
MYD03.A2014041.2010.061.2018051153130.hdf
MYD03.A2014041.2015.061.2018051153327.hdf
MYD03.A2014041.2020.061.2018051153254.hdf
MYD03.A2014041.2025.061.2018051153025.hdf
MYD03.A2014041.2030.061.2018051153531.hdf
MYD03.A2014041.2035.061.2018051153238.hdf
MYD03.A2014041.2040.061.2018051153038.hdf
MYD03.A2014041.2045.061.2018051152957.hdf
MYD03.A2014041.2050.061.2018051152909.hdf
MYD03.A2014041.2055.061.2018051153114.hdf
MYD03.A2014041.2100.061.2018051153301.hdf
MYD03.A2014041.2105.061.2018051153316.hdf
MYD03.A2014041.2110.061.2018051153314.hdf
MYD03.A2014041.2115.061.2018051153702.hdf
MYD03.A2014041.2120.061.2018051154808.hdf
MYD03.A2014041.2125.061.2018051153353.hdf
MYD03.A2014041.2130.061.2018051153342.hdf
MYD03.A2014041.2135.061.2018051153

MYD03.A2014042.1155.061.2018051153134.hdf
MYD03.A2014042.1200.061.2018051153415.hdf
MYD03.A2014042.1205.061.2018051153812.hdf
MYD03.A2014042.1210.061.2018051154310.hdf
MYD03.A2014042.1215.061.2018051153921.hdf
MYD03.A2014042.1220.061.2018051153648.hdf
MYD03.A2014042.1225.061.2018051153545.hdf
MYD03.A2014042.1230.061.2018051153733.hdf
MYD03.A2014042.1235.061.2018051153336.hdf
MYD03.A2014042.1240.061.2018051153811.hdf
MYD03.A2014042.1245.061.2018051153546.hdf
MYD03.A2014042.1250.061.2018051153529.hdf
MYD03.A2014042.1255.061.2018051153904.hdf
MYD03.A2014042.1300.061.2018051154112.hdf
MYD03.A2014042.1305.061.2018051153345.hdf
MYD03.A2014042.1310.061.2018051153320.hdf
MYD03.A2014042.1315.061.2018051153629.hdf
MYD03.A2014042.1320.061.2018051153211.hdf
MYD03.A2014042.1325.061.2018051153203.hdf
MYD03.A2014042.1330.061.2018051153344.hdf
MYD03.A2014042.1335.061.2018051153209.hdf
MYD03.A2014042.1340.061.2018051153551.hdf
MYD03.A2014042.1345.061.2018051154105.hdf
MYD03.A2014042.1350.061.2018051154

MYD03.A2014043.0410.061.2018051153516.hdf
MYD03.A2014043.0415.061.2018051153517.hdf
MYD03.A2014043.0420.061.2018051153838.hdf
MYD03.A2014043.0425.061.2018051153602.hdf
MYD03.A2014043.0430.061.2018051153805.hdf
MYD03.A2014043.0435.061.2018051154533.hdf
MYD03.A2014043.0440.061.2018051154853.hdf
MYD03.A2014043.0445.061.2018051154031.hdf
MYD03.A2014043.0450.061.2018051154006.hdf
MYD03.A2014043.0455.061.2018051154020.hdf
MYD03.A2014043.0500.061.2018051153618.hdf
MYD03.A2014043.0505.061.2018051153616.hdf
MYD03.A2014043.0510.061.2018051154035.hdf
MYD03.A2014043.0515.061.2018051153812.hdf
MYD03.A2014043.0520.061.2018051153613.hdf
MYD03.A2014043.0525.061.2018051154058.hdf
MYD03.A2014043.0530.061.2018051154211.hdf
MYD03.A2014043.0535.061.2018051153827.hdf
MYD03.A2014043.0540.061.2018051153601.hdf
MYD03.A2014043.0545.061.2018051153928.hdf
MYD03.A2014043.0550.061.2018051153439.hdf
MYD03.A2014043.0555.061.2018051153447.hdf
MYD03.A2014043.0600.061.2018051153736.hdf
MYD03.A2014043.0605.061.2018051153

### Download Caliop 1km Cloud Layer from Icare server <a class="anchor" id="download_caliop_1km_cloud_layer"></a>

### Download Caliop 5km Cloud Layer from Icare server <a class="anchor" id="download_caliop_5km_cloud_layer"></a>

### Download CloudSat 2B-GEOPROF from Icare server <a class="anchor" id="download_cloudsat_geoprof"></a>

In [18]:
# test 1 day

year = 2014
month = 2
day = 1

os.system( 'mkdir {}CLOUDSAT'.format(path_to_media) )
os.system( 'mkdir {}CLOUDSAT/2B-GEOPROF'.format(path_to_media) )
os.system( 'mkdir {}CLOUDSAT/2B-GEOPROF/{:04d}'.format(path_to_media,year) )
os.system( 'mkdir {}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}'.format(path_to_media,year,year,month,day) )

0

In [20]:
%%time

# test download 1 day

ftp = FTP('ftp.icare.univ-lille1.fr')
ftp.login(icare_username,icare_password)

directory = '/DATA/LIENS/CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/'.format(year,year,month,day)

ftp.cwd(directory)

yyy = []
ftp.retrlines('NLST', yyy.append)

for j in np.arange(len(yyy)):
    file_name = yyy[j]
    print(file_name) 
    if not path.exists('{}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)):
        ftp.retrbinary('RETR ' + file_name, open(file_name, 'wb').write)
        command = 'mv {} {}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(file_name,path_to_media,year,year,month,day,file_name) 
        os.system(command)

ftp.close()

2014032160924_41309_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032192710_41311_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032210603_41312_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032111246_41306_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032143031_41308_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032224457_41313_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032075459_41304_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032125139_41307_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032174817_41310_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032043713_41302_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032093353_41305_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032011927_41300_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032025820_41301_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032061606_41303_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
CPU times: user 397 ms, sys: 641 ms, total: 1.04 s
Wall time: 34 s


In [30]:
%%time

year = 2014
month = 2

for day in range(monthrange(year, month)[1]):
    
    day = day + 1

    d0 = date(year, 1, 1)
    d1 = date(year, month, day)
    delta = d1 - d0

    count_of_day = delta.days + 1
        
    os.system( 'mkdir {}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}'.format(path_to_media,year,year,month,day) )

    try:

        ftp = FTP('ftp.icare.univ-lille1.fr')
        ftp.login(icare_username,icare_password)

        directory = '/DATA/LIENS/CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/'.format(year,year,month,day)

        ftp.cwd(directory)

        yyy = []
        ftp.retrlines('NLST', yyy.append)

        for j in np.arange(len(yyy)):
            file_name = yyy[j]
            print(file_name) 
            if not path.exists('{}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(path_to_media,year,year,month,day,file_name)):
                ftp.retrbinary('RETR ' + file_name, open(file_name, 'wb').write)
                command = 'mv {} {}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/{}'.format(file_name,path_to_media,year,year,month,day,file_name) 
                os.system(command)

        ftp.close()    

    except:

        pass
    
    print('All files from day {} have been downloaded'.format(day))

2014032160924_41309_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032192710_41311_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032210603_41312_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032111246_41306_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032143031_41308_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032224457_41313_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032075459_41304_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032125139_41307_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032174817_41310_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032043713_41302_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032093353_41305_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032011927_41300_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032025820_41301_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014032061606_41303_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
All files from day 1 have been downloaded
2014033151346_41323_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014033034135_41316_CS_2B-GEOPROF_GRANULE_P

2014040020846_41417_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014040034739_41418_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
All files from day 9 have been downloaded
2014041174156_41441_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041060946_41434_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041192049_41442_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041223835_41444_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041043053_41433_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041074839_41435_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041092732_41436_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041142411_41439_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041124518_41438_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041205942_41443_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041011307_41431_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041025200_41432_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041110625_41437_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014041160303_41440_CS_2B-GEOPROF_GRANULE_P

All files from day 18 have been downloaded
2014050155638_41571_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050105959_41568_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050173531_41572_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050205318_41574_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050223211_41575_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050074212_41566_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050123852_41569_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050141745_41570_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050191425_41573_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014050092105_41567_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
All files from day 19 have been downloaded
2014051213636_41589_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014051082531_41581_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014051163956_41586_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014051032851_41578_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf
2014051114317_41583_CS_2B-GEOPROF_GRANULE_P1_R05_E06_F00.hdf

### Create a function to co-locate a MODIS granule with Caliop and CloudSat <a class="anchor" id="colocation_function"></a>

In [7]:
def broadcasting_based_lng_lat(data1, data2):
    # data1, data2 are the data arrays with 2 cols and they hold
    # lat., lng. values in those cols respectively
    data1 = np.deg2rad(data1)                     
    data2 = np.deg2rad(data2)                     

    lat1 = data1[:,0]                     
    lng1 = data1[:,1]         

    lat2 = data2[:,0]                     
    lng2 = data2[:,1]         

    diff_lat = lat1[:,None] - lat2
    diff_lng = lng1[:,None] - lng2
    d = np.sin(diff_lat/2)**2 + np.cos(lat1[:,None])*np.cos(lat2) * np.sin(diff_lng/2)**2
    
    return 2 * 6371 * np.arcsin(np.sqrt(d))

In [11]:
%%time

def get_modis_caliop_cloudsat_colocated_indexes(modis_file,caliop_file,cloudsat_file):
   
    modis_latitude = modis_file.select('Latitude')
    modis_longitude = modis_file.select('Longitude')
 
    modis_latitude_data = modis_latitude.get()
    modis_longitude_data = modis_longitude.get()

    modis_latitude_attributes = modis_latitude.attributes()
    #pprint.pprint(modis_latitude_attributes )

    modis_longitude_attributes = modis_longitude.attributes()
    #pprint.pprint(modis_longitude_attributes )

    modis_granule_shape = modis_latitude_data.shape
    modis_dim_along_track = modis_granule_shape[0] - 1
    #print('modis_dim_along_track ',modis_dim_along_track)

    #
    # CALIOP
    #
    
    caliop_latitude = caliop_file.select('Latitude')
    caliop_longitude = caliop_file.select('Longitude')   

    caliop_latitude_data = caliop_latitude.get()
    caliop_longitude_data = caliop_longitude.get()
    
    caliop_latitude_attributes = caliop_latitude.attributes()
    #pprint.pprint(caliop_latitude_attributes )

    caliop_longitude_attributes = caliop_longitude.attributes()
    #pprint.pprint(caliop_longitude_attributes )
    
    # Find first colocated pixel
    
    modis_pt = np.stack((modis_latitude_data[0,:],modis_longitude_data[0,:]), axis=-1)

    caliop_pt = np.stack((caliop_latitude_data[:,0],caliop_longitude_data[:,0]), axis=-1)

    #d = cdist(caliop_pt,modis_pt)
    d = broadcasting_based_lng_lat(caliop_pt,modis_pt)

    d_min = np.argwhere(d==d.min())

    CALIOP_pt1_idx = d_min[0][0]
    MODIS_pt1_idx = d_min[0][1]    
    
    # Find second colocated pixel
        
    modis_pt = np.stack((modis_latitude_data[modis_dim_along_track,:],modis_longitude_data[modis_dim_along_track,:]), axis=-1)

    caliop_pt = np.stack((caliop_latitude_data[:,0],caliop_longitude_data[:,0]), axis=-1)

    #d = cdist(caliop_pt,modis_pt)
    d = broadcasting_based_lng_lat(caliop_pt,modis_pt)

    d_min = np.argwhere(d==d.min())

    CALIOP_pt2_idx = d_min[0][0]
    MODIS_pt2_idx = d_min[0][1]        
    
    # Find all MODIS-CALIOP colocated pixels
        
    MODIS_min_idx = min(MODIS_pt1_idx,MODIS_pt2_idx) - 2
    MODIS_max_idx = max(MODIS_pt1_idx,MODIS_pt2_idx) + 2

    lat = modis_latitude_data[:,MODIS_min_idx:MODIS_max_idx]
    lat = lat.ravel()

    long = modis_longitude_data[:,MODIS_min_idx:MODIS_max_idx]
    long = long.ravel()

    modis_pt = np.stack((lat,long), axis=-1)

    caliop_pt = np.stack((caliop_latitude_data[CALIOP_pt1_idx:CALIOP_pt2_idx,0],
                          caliop_longitude_data[CALIOP_pt1_idx:CALIOP_pt2_idx,0]), axis=-1)  

    #d = cdist(caliop_pt,modis_pt)
    d = broadcasting_based_lng_lat(caliop_pt,modis_pt)

    res = np.argmin(d, axis=1)

    res = np.unravel_index(res,(modis_granule_shape[0], (MODIS_max_idx-MODIS_min_idx)))

    modis_colocated_idx_dim_0 = res[0]
    modis_colocated_idx_dim_1 = res[1]

    modis_colocated_idx_dim_1 = modis_colocated_idx_dim_1 + MODIS_min_idx
    
    caliop_idx = [i for i in range(CALIOP_pt1_idx,CALIOP_pt2_idx)] 
    
    #
    # CLOUDSAT
    #
    
    vs = cloudsat_file.vstart()

    vdata_lat = vs.attach('Latitude')
    vdata_long = vs.attach('Longitude')

    cloudsat_lat = np.array( vdata_lat[:] )
    cloudsat_long = np.array( vdata_long[:] )
        
    # Find First Colocated Pixel

    modis_pt = np.stack((modis_latitude_data[0,:],modis_longitude_data[0,:]), axis=-1)

    cloudsat_pt = np.stack((cloudsat_lat[:,0],cloudsat_long[:,0]), axis=-1)

    d = cdist(cloudsat_pt,modis_pt)

    #print(d.shape)

    #print(d.min())

    d_min = np.argwhere(d==d.min())

    CLOUDSAT_pt1_idx = d_min[0][0]
    MODIS_pt1_idx = d_min[0][1]
    
    #print('CLOUDSAT_pt1_idx', CLOUDSAT_pt1_idx)
    
    #----- find end pixel -----#

    modis_pt = np.stack((modis_latitude_data[modis_dim_along_track,:],modis_longitude_data[modis_dim_along_track,:]), axis=-1)

    cloudsat_pt = np.stack((cloudsat_lat[:,0],cloudsat_long[:,0]), axis=-1)

    d = cdist(cloudsat_pt,modis_pt)

    d_min = np.argwhere(d==d.min())

    CLOUDSAT_pt2_idx = d_min[0][0]
    MODIS_pt2_idx = d_min[0][1]
    
    #----- find all pixels -----#
    
    caliop_pt = np.stack((caliop_latitude_data[CALIOP_pt1_idx:CALIOP_pt2_idx,0],
                          caliop_longitude_data[CALIOP_pt1_idx:CALIOP_pt2_idx,0]), axis=-1)

    cloudsat_pt = np.stack((cloudsat_lat[CLOUDSAT_pt1_idx:CLOUDSAT_pt2_idx,0],
                          cloudsat_long[CLOUDSAT_pt1_idx:CLOUDSAT_pt2_idx,0]), axis=-1)
    
    #print('cloudsat_pt',cloudsat_pt.shape,CLOUDSAT_pt1_idx,CLOUDSAT_pt2_idx)
    
    d = cdist(caliop_pt,cloudsat_pt)

    #print(d.shape)

    res = np.argmin(d, axis=1)
    
    cloudsat_idx = res+CLOUDSAT_pt1_idx
    
    data = np.array([modis_colocated_idx_dim_0,modis_colocated_idx_dim_1,caliop_idx,cloudsat_idx])
    data = data.transpose()
    
    return data

# Testing

#print('Test Results: \n')

#modis_file = SD('./inputs/MYD03.A2008008.1420.006.2012066144951.hdf', SDC.READ)
#caliop_file = SD('./inputs/CAL_LID_L2_01kmCLay-ValStage1-V3-01.2008-01-08T13-55-27ZD.hdf', SDC.READ)
#cloudsat_file = HDF('./inputs/2008008133740_09032_CS_2B-CLDCLASS-LIDAR_GRANULE_P_R04_E02.hdf', SDC.READ) 

#get_modis_caliop_cloudsat_colocated_indexes(modis_file,caliop_file,cloudsat_file)

Test Results: 

CPU times: user 3.48 s, sys: 1.54 s, total: 5.02 s
Wall time: 5.04 s


array([[    0,   495, 10568, 16394],
       [    1,   495, 10569, 16394],
       [    2,   495, 10570, 16395],
       ...,
       [ 2036,   482, 12590, 18274],
       [ 2037,   482, 12591, 18274],
       [ 2038,   482, 12592, 18275]])

### Create a HDF5 file to store co-located pixels for a single day <a class="anchor" id="colocation_function"></a>

In [36]:
year = 2014
month = 2
day = 1

caliop_1km_list = glob.glob( '{}CALIOP/01kmCLay/{:04d}/{:04d}_{:02d}_{:02d}/*hdf'.format(path_to_media,year,year,month,day) )
myd03_list = glob.glob('{}MODIS/MYD03/{:04d}/{:04d}_{:02d}_{:02d}/*hdf'.format(path_to_media,year,year,month,day))
cloudsat_cldprop_list = glob.glob( '{}CLOUDSAT/2B-GEOPROF/{:04d}/{:04d}_{:02d}_{:02d}/*hdf'.format(path_to_media,year,year,month,day) )

print(len(caliop_1km_list))
print(len(myd03_list))
print(len(cloudsat_cldprop_list))

29
288
14


In [37]:
%%time

#columns = ['myd03_name', 'cal_1km_file', 'cloudsat_2B_cldclass_lidar_file']

if path.exists('modis_caliop_cloudsat_colocated_db_{:04d}_{:02d}.hdf5'.format(year,month)):
    with pd.HDFStore('modis_caliop_cloudsat_colocated_db_{:04d}_{:02d}.hdf5'.format(year,month)) as store:
        match_files = store['match_files']
        results = match_files.to_dict()
else:
    results = {}

store = pd.HDFStore('modis_caliop_cloudsat_colocated_db_{:04d}_{:02d}.hdf5'.format(year,month))    
    
count = 1

for idx,myd03_path in enumerate(myd03_list):
#for idx,myd03_path in enumerate(myd03_list[50:55]):
    
    myd03_name = myd03_path.split('/')[-1]
        
    hm = myd03_name.split('.')[2]

    modis_granule_hour = int(hm[0:2])
    modis_granule_minute = int(hm[2:4])

    cal_1km_file = 'None'
    cloudsat_2B_cldclass_lidar_file = 'None'
    
    #----- Caliop 01km_clay -----#
    
    time_min = 200

    for caliop_01km_granule_name in caliop_1km_list:

        caliop_granule_name = caliop_01km_granule_name.split('/')[-1]
        
        caliop_re = re.compile("(.*)T(.*)-(.*)-(.*).hdf")

        obj = caliop_re.search(caliop_granule_name)
        
        caliop_granule_hour = int(obj.group(2))
        caliop_granule_minute = int(obj.group(3))

        a = datetime.datetime(year, month, day, modis_granule_hour, modis_granule_minute, 0) 
        b = datetime.datetime(year, month, day, caliop_granule_hour, caliop_granule_minute, 0)
        
        time_delta = a-b         
        
        total_seconds = time_delta.total_seconds()
        minutes = total_seconds/60
        
        #print('datetime diff ----> ',time_delta,caliop_granule_name,minutes)
        
        diff = int(modis_granule_hour) * 60 + int(modis_granule_minute) \
                - ( int(caliop_granule_hour) * 60 + int(caliop_granule_minute) )

        if diff < time_min and diff > 0 :
            time_min = diff
            cal_1km_file = caliop_granule_name
            cal_1km_w_path = caliop_01km_granule_name
    
    #----- cloudsat (2B CLDCLASS-LIDAR) -----#
    
    cloudsat_minutes_delta_list = []
    
    for cloudsat_file in cloudsat_cldprop_list:
        
        cloudsat_file_name = cloudsat_file.split('/')[-1]
        
        cloudsat_granule_hour = int(cloudsat_file_name[7:9])
        cloudsat_granule_minute = int(cloudsat_file_name[9:11])
        cloudsat_granule_second = int(cloudsat_file_name[11:13] )

        a = datetime.datetime(year, month, day, modis_granule_hour, modis_granule_minute, 0) 
        b = datetime.datetime(year, month, day, cloudsat_granule_hour, cloudsat_granule_minute, 0)
        
        time_delta = a-b         
        
        total_seconds = time_delta.total_seconds()
        minutes = total_seconds/60
        
        cloudsat_minutes_delta_list.append(minutes)
        
    cloudsat_minutes_positive_delta_list = [i for i in cloudsat_minutes_delta_list if i > 0]

    if cloudsat_minutes_positive_delta_list:    
        cloudsat_minutes_delta_min_idx = cloudsat_minutes_delta_list.index(min(cloudsat_minutes_positive_delta_list))
        cloudsat_path = cloudsat_cldprop_list[cloudsat_minutes_delta_min_idx]
        cloudsat_filename = cloudsat_path.split('/')[-1]
    else:
        cloudsat_path = 'None'
        cloudsat_filename = 'None'
    
    #file_found_list = [myd03_name,cal_1km_file,cloudsat_filename]
    
    file_found_dic = {}
    file_found_dic['modis_file'] = myd03_name
    file_found_dic['caliop_file'] = cal_1km_file
    file_found_dic['cloudsat_file'] = cloudsat_filename
    
    #results['{:02d}_{:02d}_{:02d}'.format(day,modis_granule_hour,modis_granule_minute)] = file_found_list
    results['{:02d}_{:02d}_{:02d}'.format(day,modis_granule_hour,modis_granule_minute)] = file_found_dic
    
    if cal_1km_file != 'None' and cloudsat_path != 'None':
        
        try:

            modis_file = SD(myd03_path, SDC.READ)
            caliop_file = SD(cal_1km_w_path, SDC.READ)
            cloudsat_file = HDF(cloudsat_path, SDC.READ) 

            data = get_modis_caliop_cloudsat_colocated_indexes(modis_file,caliop_file,cloudsat_file)

            df = pd.DataFrame(data=data,columns=['modis_indexes_0','modis_indexes_1','caliop_1km_indexes','cloudsat_indexes'])

            store.put('colocated_indexes_{:02d}_{:02d}_{:02d}'.format(day,modis_granule_hour,modis_granule_minute), df)

            print(count, ' / 288 :', myd03_name, ' done !')

            count += 1
        
        except:
            
            pass
        
df = pd.DataFrame(data=results, index=['modis_file','caliop_file','cloudsat_file'])

store.put('match_files', df)

store.close()

1  / 288 : MYD03.A2014032.0120.061.2018051150857.hdf  done !
2  / 288 : MYD03.A2014032.0125.061.2018051150749.hdf  done !
3  / 288 : MYD03.A2014032.0130.061.2018051151245.hdf  done !
4  / 288 : MYD03.A2014032.0135.061.2018051151824.hdf  done !
5  / 288 : MYD03.A2014032.0140.061.2018051152726.hdf  done !
6  / 288 : MYD03.A2014032.0145.061.2018051151503.hdf  done !
7  / 288 : MYD03.A2014032.0150.061.2018051151229.hdf  done !
8  / 288 : MYD03.A2014032.0155.061.2018051151013.hdf  done !
9  / 288 : MYD03.A2014032.0200.061.2018051150939.hdf  done !
10  / 288 : MYD03.A2014032.0205.061.2018051151029.hdf  done !
11  / 288 : MYD03.A2014032.0210.061.2018051151010.hdf  done !
12  / 288 : MYD03.A2014032.0215.061.2018051151056.hdf  done !
13  / 288 : MYD03.A2014032.0220.061.2018051151203.hdf  done !
14  / 288 : MYD03.A2014032.0225.061.2018051151337.hdf  done !
15  / 288 : MYD03.A2014032.0230.061.2018051151947.hdf  done !
16  / 288 : MYD03.A2014032.0235.061.2018051151053.hdf  done !
17  / 288 : MYD03

133  / 288 : MYD03.A2014032.1245.061.2018051151125.hdf  done !
134  / 288 : MYD03.A2014032.1255.061.2018051151026.hdf  done !
135  / 288 : MYD03.A2014032.1300.061.2018051151303.hdf  done !
136  / 288 : MYD03.A2014032.1305.061.2018051151614.hdf  done !
137  / 288 : MYD03.A2014032.1310.061.2018051152335.hdf  done !
138  / 288 : MYD03.A2014032.1315.061.2018051151843.hdf  done !
139  / 288 : MYD03.A2014032.1320.061.2018051151532.hdf  done !
140  / 288 : MYD03.A2014032.1325.061.2018051151425.hdf  done !
141  / 288 : MYD03.A2014032.1330.061.2018051151152.hdf  done !
142  / 288 : MYD03.A2014032.1335.061.2018051151135.hdf  done !
143  / 288 : MYD03.A2014032.1340.061.2018051151703.hdf  done !
144  / 288 : MYD03.A2014032.1345.061.2018051151449.hdf  done !
145  / 288 : MYD03.A2014032.1350.061.2018051151326.hdf  done !
146  / 288 : MYD03.A2014032.1355.061.2018051151450.hdf  done !
147  / 288 : MYD03.A2014032.1400.061.2018051151942.hdf  done !
148  / 288 : MYD03.A2014032.1405.061.2018051151151.hdf 

### Read data from the HDF5 file <a class="anchor" id="read_hdf5"></a>

In [39]:
with pd.HDFStore('modis_caliop_cloudsat_colocated_db_2014_02.hdf5') as store:
    print(store.keys())
    print(len(store.keys()))

['/colocated_indexes_01_01_20', '/colocated_indexes_01_01_25', '/colocated_indexes_01_01_30', '/colocated_indexes_01_01_35', '/colocated_indexes_01_01_40', '/colocated_indexes_01_01_45', '/colocated_indexes_01_01_50', '/colocated_indexes_01_01_55', '/colocated_indexes_01_02_00', '/colocated_indexes_01_02_05', '/colocated_indexes_01_02_10', '/colocated_indexes_01_02_15', '/colocated_indexes_01_02_20', '/colocated_indexes_01_02_25', '/colocated_indexes_01_02_30', '/colocated_indexes_01_02_35', '/colocated_indexes_01_02_40', '/colocated_indexes_01_02_45', '/colocated_indexes_01_02_50', '/colocated_indexes_01_03_00', '/colocated_indexes_01_03_05', '/colocated_indexes_01_03_10', '/colocated_indexes_01_03_15', '/colocated_indexes_01_03_20', '/colocated_indexes_01_03_25', '/colocated_indexes_01_03_30', '/colocated_indexes_01_03_35', '/colocated_indexes_01_03_40', '/colocated_indexes_01_03_45', '/colocated_indexes_01_03_50', '/colocated_indexes_01_03_55', '/colocated_indexes_01_04_00', '/coloc

In [42]:
with pd.HDFStore('modis_caliop_cloudsat_colocated_db_2014_02.hdf5') as store:
    #print(store.keys())
    print(len(store.keys()))
    match_files = store['match_files']
    colocated_indexes = store['colocated_indexes_01_02_35']

#print(match_files)
#print(match_files.head())

print(match_files.loc[:,'01_02_35'])

print(colocated_indexes.shape)
print(colocated_indexes)

263
modis_file               MYD03.A2014032.0235.061.2018051151053.hdf
caliop_file      CAL_LID_L2_01kmCLay-Standard-V4-20.2014-02-01T...
cloudsat_file    2014032011927_41300_CS_2B-GEOPROF_GRANULE_P1_R...
Name: 01_02_35, dtype: object
(2014, 4)
      modis_indexes_0  modis_indexes_1  caliop_1km_indexes  cloudsat_indexes
0                   0              736                2976             29599
1                   1              736                2977             29600
2                   2              736                2978             29601
3                   3              736                2979             29602
4                   4              736                2980             29603
...               ...              ...                 ...               ...
2009             2024              793                4985             31467
2010             2025              793                4986             31468
2011             2026              793                4987    

In [43]:
#print(match_files)
print(type(match_files))
print(match_files.head())

<class 'pandas.core.frame.DataFrame'>
                                                01_00_00  \
modis_file     MYD03.A2014032.0000.061.2018051152159.hdf   
caliop_file                                         None   
cloudsat_file                                       None   

                                                01_00_05  \
modis_file     MYD03.A2014032.0005.061.2018051151531.hdf   
caliop_file                                         None   
cloudsat_file                                       None   

                                                01_00_10  \
modis_file     MYD03.A2014032.0010.061.2018051151021.hdf   
caliop_file                                         None   
cloudsat_file                                       None   

                                                01_00_15  \
modis_file     MYD03.A2014032.0015.061.2018051151148.hdf   
caliop_file                                         None   
cloudsat_file                                       None   

In [54]:
test_to_dict = match_files.to_dict()

type(test_to_dict)

dict

In [55]:
len(test_to_dict)

14

In [56]:
test_to_dict

{'08_02_35': {'modis_file': 'MYD03.A2008190.0235.061.2018034131058.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_40': {'modis_file': 'MYD03.A2008190.0240.061.2018034131612.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_45': {'modis_file': 'MYD03.A2008190.0245.061.2018034131239.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_50': {'modis_file': 'MYD03.A2008190.0250.061.2018034131248.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_55': {'modis_file': 'MYD03.A2008190.0255

In [57]:
results

{'08_02_35': {'modis_file': 'MYD03.A2008190.0235.061.2018034131058.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_40': {'modis_file': 'MYD03.A2008190.0240.061.2018034131612.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_45': {'modis_file': 'MYD03.A2008190.0245.061.2018034131239.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_50': {'modis_file': 'MYD03.A2008190.0250.061.2018034131248.hdf',
  'caliop_file': 'CAL_LID_L2_01kmCLay-Standard-V4-20.2008-07-08T02-00-58ZD.hdf',
  'cloudsat_file': '2008190012949_11675_CS_2B-GEOPROF_GRANULE_P1_R05_E02_F00.hdf'},
 '08_02_55': {'modis_file': 'MYD03.A2008190.0255

In [58]:
test_to_dict == results

True

In [24]:
'''
store = pd.HDFStore('data.hdf5')

Missing optional dependency 'tables'.  Use pip or conda to install tables.

Fix:
    
    conda install pytables
    [Missing optional dependency 'tables'. In pandas to_hdf](https://stackoverflow.com/questions/58479748/missing-optional-dependency-tables-in-pandas-to-hdf)
    
    
  cannot set WRITEABLE flag to True of this array  
    
    conda update pytables
    
    conda upgrade --all
    
    
    [AttributeError: module 'pandas' has no attribute 'core'](https://stackoverflow.com/questions/45357826/attributeerror-module-pandas-has-no-attribute-core/45850175)
'''

"\nstore = pd.HDFStore('data.hdf5')\n\nMissing optional dependency 'tables'.  Use pip or conda to install tables.\n\nFix:\n    \n    conda install pytables\n    [Missing optional dependency 'tables'. In pandas to_hdf](https://stackoverflow.com/questions/58479748/missing-optional-dependency-tables-in-pandas-to-hdf)\n    \n    \n  cannot set WRITEABLE flag to True of this array  \n    \n    conda update pytables\n    \n    conda upgrade --all\n    \n    \n    [AttributeError: module 'pandas' has no attribute 'core'](https://stackoverflow.com/questions/45357826/attributeerror-module-pandas-has-no-attribute-core/45850175)\n"